In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

In [ ]:
# Not for batch
# Slide 2d product
class Conv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
    super(Conv2d, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.bias = bias

    self.weight = Parameter(torch.rand(out_channels, in_channels, kernel_size[0], kernel_size[1]))
    if bias:
      self.bias = Parameter(torch.rand(out_channels, ))
    else:
      self.bias = torch.zeros(out_channels, )

  def _conv2d(self, x):
    input_height, input_width = x[0].shape
    output_height = ((input_height - self.kernel_size[0]) // self.stride) + 1
    output_width = ((input_width - self.kernel_size[1]) // self.stride) + 1

    output = torch.zeros(self.out_channels, output_height, output_width)
    for channel in range(0, self.out_channels):
      for i in range(0, output_height):
        for j in range(0, output_width):
          x_i = i * self.stride
          x_j = j * self.stride

          weighted = x[:, x_i : x_i+self.kernel_size[0], x_j : x_j+self.kernel_size[1]] * self.weight[channel, :, :, :]
          weighted_sum = weighted.sum()
          output[channel][i][j] += weighted_sum
    output = output + self.bias.unsqueeze(dim=-1).unsqueeze(dim=-1)

    return output

  def forward(self, x):
    return self._conv2d(F.pad(x, (self.padding, self.padding, self.padding, self.padding)))


padding=2
stride=2
img = torch.rand(3, 10, 10)
conv_layer = Conv2d(3, 5, [3, 3], stride, padding, True)
print(conv_layer(img))
print(F.conv2d(img, conv_layer.weight.data, bias=conv_layer.bias.data, padding=padding, stride=stride))

tensor([[[1.4531, 2.0830, 2.4913, 3.5422, 2.6706, 1.9279],
         [2.7544, 5.9963, 6.4142, 7.8939, 8.1696, 4.7225],
         [3.4721, 8.0339, 7.6732, 5.9775, 6.7201, 4.5327],
         [2.7897, 7.1372, 8.5761, 7.9110, 7.2191, 4.3421],
         [3.1882, 6.3207, 7.1098, 7.7169, 6.5752, 2.9402],
         [2.9439, 5.7411, 4.5029, 5.3208, 5.1562, 2.9408]],

        [[0.8749, 2.2232, 2.3732, 2.7181, 2.1921, 2.2062],
         [2.0176, 6.2151, 6.9748, 6.6148, 8.0718, 5.1962],
         [2.2631, 7.7246, 7.6625, 5.2598, 6.4408, 4.4251],
         [1.8297, 7.4531, 7.4914, 7.2462, 6.7999, 4.6814],
         [1.7149, 6.3982, 5.6596, 6.6082, 5.8654, 3.3896],
         [1.8869, 5.8692, 3.4986, 4.4674, 4.8889, 2.9840]],

        [[0.4590, 1.3713, 1.5271, 1.6470, 1.5569, 1.3965],
         [1.4594, 5.3807, 6.0466, 6.2504, 6.8057, 4.2634],
         [1.7560, 6.9333, 5.9880, 4.4307, 5.7325, 4.5617],
         [1.6006, 6.0603, 6.5098, 6.7400, 5.2960, 4.2841],
         [1.4064, 5.9214, 4.9917, 5.9871, 5.5419, 3.

In [ ]:
# Not for batch
# Flatten the kernel and windows and then do dot product
class Conv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
    super(Conv2d, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.bias = bias

    self.weight = Parameter(torch.rand(out_channels, in_channels, kernel_size[0], kernel_size[1]))
    if bias:
      self.bias = Parameter(torch.rand(out_channels, ))
    else:
      self.bias = torch.zeros(out_channels, )

  def _conv2d(self, x):
    input_height, input_width = x[0].shape
    output_height = ((input_height - self.kernel_size[0]) // self.stride) + 1
    output_width = ((input_width - self.kernel_size[1]) // self.stride) + 1

    output = torch.zeros(self.out_channels, output_height, output_width)

    weight = self.weight.reshape(self.weight.shape[0], -1)  # flatten every kernel

    for channel in range(0, self.out_channels):
      window_stack = torch.zeros(output_height * output_width, self.in_channels * self.kernel_size[0] * self.kernel_size[1])
      for i in range(0, output_height):
        for j in range(0, output_width):
          x_i = i * self.stride
          x_j = j * self.stride

          window = x[:, x_i : x_i+self.kernel_size[0], x_j : x_j+self.kernel_size[1]]
          window = window.reshape(-1)   # flatten the window
          window_stack[i*output_height + j] = window # add the window to the window_stack
      weighted_sum = window_stack @ weight[channel] # dot product each window to the kernel
      output[channel] = weighted_sum.reshape(output_height, output_width)

    output = output + self.bias.unsqueeze(dim=-1).unsqueeze(dim=-1)
    return output

  def forward(self, x):
    return self._conv2d(F.pad(x, (self.padding, self.padding, self.padding, self.padding)))


padding=2
stride=2
img = torch.rand(3, 10, 10)
conv_layer = Conv2d(3, 5, [3, 3], stride, padding, True)
print(conv_layer(img))
print(F.conv2d(img, conv_layer.weight.data, bias=conv_layer.bias.data, padding=padding, stride=stride))

tensor([[[ 1.4270,  2.6039,  2.7978,  2.4597,  2.2008,  1.7372],
         [ 2.9344,  7.6004,  7.2254,  6.9609,  5.8994,  4.3249],
         [ 2.7976,  6.0593,  6.2952,  6.9556,  6.2254,  3.4357],
         [ 3.3958,  6.6478,  6.2360,  6.8679,  5.5337,  2.8626],
         [ 2.1689,  5.5544,  4.9760,  7.2194,  5.9433,  3.5074],
         [ 2.0495,  4.2283,  5.0333,  4.6419,  4.3606,  2.8853]],

        [[ 1.0778,  2.9724,  3.6974,  2.7337,  2.4701,  2.5158],
         [ 1.7585,  5.5037,  6.7694,  5.5176,  6.0198,  3.8737],
         [ 1.9761,  5.2306,  6.8475,  6.9044,  5.3635,  2.4086],
         [ 1.7406,  5.4427,  4.7411,  5.0916,  4.7052,  2.5696],
         [ 1.0850,  4.8542,  4.1027,  6.2888,  5.7497,  2.8120],
         [ 0.8657,  2.8196,  3.1356,  2.7571,  3.0032,  1.6076]],

        [[ 1.5295,  3.4482,  4.0657,  3.3698,  3.0910,  2.5728],
         [ 2.7892,  8.8608,  8.9145,  8.9226,  7.4301,  6.1307],
         [ 3.0756,  7.6366,  9.0439,  8.7996,  7.0920,  4.4284],
         [ 2.8042,  7

In [ ]:
# Not for batch
# pad the kernel and do dot product with the whole image
class Conv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
    super(Conv2d, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.bias = bias

    self.weight = Parameter(torch.rand(out_channels, in_channels, kernel_size[0], kernel_size[1]))
    if bias:
      self.bias = Parameter(torch.rand(out_channels, ))
    else:
      self.bias = torch.zeros(out_channels, )

  def getKernelStack(self, output_height, output_width, input_height, input_width):
    kernel_stack = torch.zeros(self.out_channels * output_height * output_width, self.in_channels * input_height * input_width)
    for channel in range(0, self.out_channels):
      for i in range(0, output_height):
        for j in range(0, output_width):
          up = i * self.stride
          left = j * self.stride

          bottom = input_height - self.kernel_size[0] - up
          right = input_width - self.kernel_size[1] - left

          kernel_stack[channel * output_height * output_width + i*output_width + j] = F.pad(self.weight[channel], (left, right, up, bottom)).reshape(-1)
    return kernel_stack

  def _conv2d(self, x):
    input_height, input_width = x[0].shape
    output_height = ((input_height - self.kernel_size[0]) // self.stride) + 1
    output_width = ((input_width - self.kernel_size[1]) // self.stride) + 1

    output = torch.zeros(self.out_channels, output_height, output_width)

    x = x.reshape(-1)  # flatten input

    kernel_stack = self.getKernelStack(output_height, output_width, input_height, input_width)
    weighted_sum = kernel_stack @ x
    output = weighted_sum.reshape(self.out_channels, output_height, output_width)

    output = output + self.bias.unsqueeze(dim=-1).unsqueeze(dim=-1)
    return output

  def forward(self, x):
    return self._conv2d(F.pad(x, (self.padding, self.padding, self.padding, self.padding)))


padding=2
stride=2
img = torch.rand(3, 10, 10)
conv_layer = Conv2d(3, 5, [3, 3], stride, padding, False)
print(conv_layer(img))
print(F.conv2d(img, conv_layer.weight.data, bias=conv_layer.bias.data, padding=padding, stride=stride))

tensor([[[ 0.5424,  2.8086,  2.0527,  2.4725,  2.2679,  1.1477],
         [ 2.3743,  6.8770,  6.5688,  7.9841,  6.8333,  4.8689],
         [ 2.5246,  7.5350,  5.6906,  7.3873,  7.1638,  4.7830],
         [ 1.8095,  7.5537,  6.7057,  6.4189,  7.3862,  5.5735],
         [ 2.4941,  6.7883,  8.0656,  8.4806,  7.8950,  5.2561],
         [ 1.6898,  4.7093,  5.0057,  5.0587,  5.1795,  2.5376]],

        [[ 0.8665,  1.8311,  1.8487,  2.2015,  2.5681,  1.1162],
         [ 2.8540,  6.6134,  6.9374,  7.7696,  6.3004,  3.3082],
         [ 3.1500,  7.9742,  7.2357,  7.4050,  6.8220,  3.6889],
         [ 3.3433,  7.3339,  5.9514,  7.0688,  8.1528,  4.4445],
         [ 3.0505,  6.8490,  7.2185,  8.4331,  8.5285,  4.5680],
         [ 1.4607,  4.0364,  4.7277,  5.0921,  4.1337,  2.5328]],

        [[ 0.2923,  1.2190,  1.0026,  1.5044,  1.3328,  0.8121],
         [ 2.3882,  6.3056,  5.4415,  7.0724,  5.4973,  4.3390],
         [ 2.3874,  6.1741,  6.7679,  6.5000,  6.4788,  4.1741],
         [ 2.4580,  6

In [ ]:
# Not for batch
# Just multiply the tranpose of kernelstach with the result of conv2d
class ConvTranspose2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
    super(ConvTranspose2d, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.bias = bias

    self.weight = Parameter(torch.rand(out_channels, in_channels, kernel_size[0], kernel_size[1]))
    if bias:
      self.bias = Parameter(torch.rand(in_channels, ))
    else:
      self.bias = torch.zeros(in_channels, )

  def getKernelStack(self, output_height, output_width, input_height, input_width):
    kernel_stack = torch.zeros(self.out_channels * output_height * output_width, self.in_channels * input_height * input_width)
    for channel in range(0, self.out_channels):
      for i in range(0, output_height):
        for j in range(0, output_width):
          up = i * self.stride
          left = j * self.stride

          bottom = input_height - self.kernel_size[0] - up
          right = input_width - self.kernel_size[1] - left

          kernel_stack[channel * output_height * output_width + i*output_width + j] = F.pad(self.weight[channel], (left, right, up, bottom)).reshape(-1)
    return kernel_stack

  def _conv_transpose2d(self, x):
    output_height, output_width = x[0].shape
    input_height = (output_height - 1) * self.stride + self.kernel_size[0]
    input_width = (output_width - 1) * self.stride + self.kernel_size[1]

    output = torch.zeros(self.in_channels, input_height, input_width)

    x = x.reshape(-1)  # flatten input

    kernel_stack = self.getKernelStack(output_height, output_width, input_height, input_width)
    weighted_sum = kernel_stack.T @ x
    output = weighted_sum.reshape(self.in_channels, input_height, input_width)

    output = output + self.bias.unsqueeze(dim=-1).unsqueeze(dim=-1)
    return output

  def forward(self, x):
    return self._conv_transpose2d(F.pad(x, (self.padding, self.padding, self.padding, self.padding)))


padding=0
stride=1
img = torch.rand(2, 6, 6)
convTranspose_layer = ConvTranspose2d(3, 2, [3, 3], stride, padding, True)
print(convTranspose_layer(img))
print(F.conv_transpose2d(img, convTranspose_layer.weight.data, bias=convTranspose_layer.bias.data, padding=padding, stride=stride))

tensor([[[0.9563, 1.8329, 2.4744, 2.3191, 1.9618, 2.4013, 1.6376, 0.7762],
         [1.4549, 2.8377, 4.4750, 4.3264, 4.1578, 4.4060, 3.9030, 1.6700],
         [1.4158, 3.5892, 4.7237, 5.3647, 6.0045, 6.1772, 4.9488, 2.4871],
         [1.3834, 2.9721, 4.8428, 4.5803, 5.3771, 6.8222, 4.6053, 2.4863],
         [1.5474, 3.2271, 5.0105, 5.3270, 5.6980, 6.0726, 4.7158, 2.4186],
         [1.9194, 4.2814, 5.2582, 4.8310, 4.6661, 5.2510, 4.3296, 2.5684],
         [1.4018, 2.8031, 4.2586, 3.2693, 3.1328, 3.0245, 3.1548, 1.8818],
         [0.7989, 1.5837, 1.4506, 1.4636, 1.1113, 1.3969, 1.4932, 0.7837]],

        [[0.9349, 1.4394, 2.1490, 2.1387, 1.9601, 2.5860, 2.0750, 1.7306],
         [1.6959, 2.4759, 3.9531, 4.0506, 4.2698, 4.0064, 3.6522, 2.1594],
         [2.1406, 3.9426, 4.8002, 5.3252, 6.5377, 6.4170, 5.5258, 2.7554],
         [1.6445, 3.0900, 4.9248, 5.0682, 6.7144, 6.5434, 4.3499, 3.0820],
         [1.8923, 2.8134, 4.7338, 5.3393, 6.2513, 6.1442, 4.8495, 3.0208],
         [2.2771, 3.801

In [ ]:
# Not for batch
# add groups and dilation
class Conv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, dilation=1, bias=True):
    super(Conv2d, self).__init__()

    assert(in_channels % groups == 0)
    assert(out_channels % groups == 0)

    self.groups = groups
    self.dilation = dilation
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.bias = bias

    self.weight = Parameter(torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1]))
    if bias:
      self.bias = Parameter(torch.rand(out_channels, ))
    else:
      self.bias = torch.zeros(out_channels, )

  def getKernelStack(self, output_height, output_width, input_height, input_width, dilated_kernel_size, dilated_weight):
    kernel_stack = torch.zeros(self.out_channels//self.groups * output_height * output_width, self.in_channels//self.groups * input_height * input_width)
    for channel in range(0, self.out_channels//self.groups):
      for i in range(0, output_height):
        for j in range(0, output_width):
          up = i * self.stride
          left = j * self.stride

          bottom = input_height - dilated_kernel_size[0] - up
          right = input_width - dilated_kernel_size[1] - left

          kernel_stack[channel * output_height * output_width + i*output_width + j] = F.pad(dilated_weight[channel], (left, right, up, bottom)).reshape(-1)
    return kernel_stack

  def _conv2d(self, x):
    group_weights = self.weight.reshape(self.groups, self.weight.shape[0]//self.groups, self.weight.shape[1], self.weight.shape[2], self.weight.shape[3])
    dilated_kernel_size = (torch.tensor(self.kernel_size) - 1) * self.dilation + 1
    group_dilated_weights = torch.zeros(group_weights.shape[0], group_weights.shape[1], group_weights.shape[2], dilated_kernel_size[0], dilated_kernel_size[1])
    group_dilated_weights[:, :, :, 0::self.dilation, 0::self.dilation] = group_weights

    input_height, input_width = x[0].shape
    output_height = ((input_height - dilated_kernel_size[0]) // self.stride) + 1
    output_width = ((input_width - dilated_kernel_size[1]) // self.stride) + 1
    output = torch.zeros(self.out_channels, output_height, output_width)

    for group in range(0, self.groups):
      group_output = torch.zeros(self.out_channels//self.groups, output_height, output_width)

      group_x = x[group*self.in_channels//self.groups : (group+1)*self.in_channels//self.groups].reshape(-1)  # flatten input

      kernel_stack = self.getKernelStack(output_height, output_width, input_height, input_width, dilated_kernel_size, group_dilated_weights[group])
      weighted_sum = kernel_stack @ group_x
      group_output = weighted_sum.reshape(self.out_channels//self.groups, output_height, output_width)
      output[group*self.out_channels//self.groups : (group+1)*self.out_channels//self.groups] = group_output
    output = output + self.bias.unsqueeze(dim=-1).unsqueeze(dim=-1)
    return output

  def forward(self, x):
    return self._conv2d(F.pad(x, (self.padding, self.padding, self.padding, self.padding)))


padding=2
stride=2
groups = 2
dilation = 2
img = torch.rand(6, 10, 10)
conv_layer = Conv2d(6, 8, [3, 3], stride, padding, groups, dilation, True)
print(conv_layer(img))
print(F.conv2d(img, conv_layer.weight.data, bias=conv_layer.bias.data, padding=padding, stride=stride, groups=groups, dilation=dilation))

tensor([[[2.8316, 5.0937, 5.1571, 5.4896, 4.2682],
         [3.7342, 7.3872, 7.7916, 9.3406, 6.4387],
         [3.9233, 6.7700, 6.7458, 8.8428, 5.5295],
         [4.9627, 7.0903, 7.0385, 7.9354, 5.6690],
         [4.3209, 5.3555, 4.8767, 4.8104, 3.5950]],

        [[2.5928, 4.3266, 4.7514, 5.4068, 3.3551],
         [4.1160, 7.6977, 8.6994, 9.6977, 6.0483],
         [4.2800, 5.8484, 7.4813, 7.4894, 5.8215],
         [5.7874, 7.9622, 8.0109, 7.8491, 5.8200],
         [4.2431, 5.4132, 4.9332, 4.0053, 2.6362]],

        [[2.7013, 4.9302, 5.5129, 5.3269, 3.9026],
         [4.7490, 7.3597, 9.0322, 8.8845, 5.0088],
         [4.2753, 5.9088, 7.4032, 8.4777, 5.0179],
         [5.7142, 7.7831, 8.8974, 8.0653, 5.3302],
         [5.4312, 5.7792, 6.0276, 4.9845, 3.4999]],

        [[1.8495, 2.9027, 3.6607, 4.5729, 3.2773],
         [2.8428, 5.1842, 6.4025, 8.1739, 5.5141],
         [2.3623, 4.8496, 5.9684, 6.3230, 4.9481],
         [3.8032, 6.4507, 7.3733, 6.4317, 5.2094],
         [3.5574, 4.3753,